In [4]:
w = 2.0 # 
h = 1.0 
offset = 0.1
racing_setup = { 0: [[0, 1, h/2+offset], [-w/2, 1, h+offset], [w/2, 1, h+offset], [-w/2, 1, offset], [w/2, 1, offset]],
                            1: [[0.5, 3, h/2+offset], [-w/2+0.5, 3, h+offset], [w/2+0.5, 3, h+offset], [-w/2+0.5, 3, offset], [w/2+0.5, 3, offset]],
                            2: [[-0.5, 5, h/2+offset], [-w/2-0.5, 5, h+offset], [w/2-0.5, 5, h+offset], [-w/2-0.5, 5, offset], [w/2-0.5, 5, offset]],
                            3: [[0.5, 7, h/2+offset], [-w/2+0.5, 7, h+offset], [w/2+0.5, 7, h+offset], [-w/2+0.5, 7, offset], [w/2+0.5, 7, offset]],
                            4: [[1.0, 9, h/2+offset], [-w/2+1.0, 9, h+offset], [w/2+1.0, 9, h+offset], [-w/2+1.0, 9, offset], [w/2+1.0, 9, offset]]
                            }
passing_flag = [False for _ in range(len(racing_setup.keys()))]


In [5]:
passing_flag

[False, False, False, False, False]

In [ ]:
for i, key in enumerate(racing_setup.keys()):
    if passing_flag[i]:
        continue
    if racing_setup[key][1][0] < pos[0] < racing_setup[key][2][0] and \
        racing_setup[key][1][1] - 0.5 < pos[1] < racing_setup[key][2][1] + 0.5 and \
        racing_setup[key][4][2] < pos[2] < racing_setup[key][1][2]:
        passing_flag[i] = True
        passing = True
        break